<a href="https://colab.research.google.com/github/vibhuverma17/COACH/blob/main/Graph_Intrinsic_Eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Graph-based Libraries
!pip install networkx               # Graph manipulation
!pip install node2vec               # Graph embeddings using Node2Vec
!pip install torch-geometric        # Geometric deep learning library

# Machine Learning Libraries
!pip install scikit-learn           # General machine learning tools
!pip install tensorflow             # Deep learning framework
!pip install torch                  # PyTorch framework

# Visualization Libraries
!pip install matplotlib             # Data visualization

# Knowledge Graphs and Optimization Libraries
!pip install pykeen                 # Knowledge graph embeddings
!pip install pykeops                # Optimized computation for large-scale problems

# Additional Libraries
!pip install numpy                  # Numerical computations
!pip install hdbscan                 # Hierarchical Density-Based Spatial Clustering of Applications with Noise (HDBSCAN)

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, NMF
from sklearn.preprocessing import normalize, StandardScaler
from node2vec import Node2Vec
import hdbscan
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.utils import from_networkx, to_networkx
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, VGAE, TransformerConv, global_mean_pool
from torch_geometric.datasets import Planetoid, Reddit
from pykeops.torch import LazyTensor

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics import silhouette_score


dataset = Planetoid(root='./data/CiteSeer', name='CiteSeer')
data = dataset[0]

G = to_networkx(data, to_undirected=True)

print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

subgraph = nx.subgraph(G, list(G.nodes))
plt.figure(figsize=(10, 10))
nx.draw(subgraph, node_size=10)
plt.show()



# Function for clustering and evaluation
def cluster_and_evaluate(embeddings, graph):
    # Normalize embeddings for better clustering performance
    normalized_embeddings = normalize(embeddings)

    # Clustering: K-Means
    kmeans = KMeans(n_clusters=5, random_state=42)
    kmeans_labels = kmeans.fit_predict(normalized_embeddings)

    # Clustering: Spectral Clustering
    spectral = SpectralClustering(n_clusters=5, affinity='nearest_neighbors', random_state=42)
    spectral_labels = spectral.fit_predict(normalized_embeddings)

    # Calculate Silhouette Score
    silhouette_kmeans = silhouette_score(normalized_embeddings, kmeans_labels)
    silhouette_spectral = silhouette_score(normalized_embeddings, spectral_labels)

    # Modularity Calculation
    def calculate_modularity(graph, labels):
        # Create a dictionary to group nodes by their labels
        communities = {i: [] for i in set(labels)}
        for node, label in enumerate(labels):
            communities[label].append(node)

        # Convert to a list of sets for modularity calculation
        community_list = [set(community) for community in communities.values()]

        # Check if the communities form a valid partition
        all_nodes = set(graph.nodes)
        covered_nodes = set().union(*community_list)
        if all_nodes != covered_nodes:
            raise ValueError("Generated communities do not cover all nodes in the graph.")

        return nx.algorithms.community.modularity(graph, community_list)

    modularity_kmeans = calculate_modularity(graph, kmeans_labels)
    modularity_spectral = calculate_modularity(graph, spectral_labels)

    return {
        'Silhouette KMeans': silhouette_kmeans,
        'Silhouette Spectral': silhouette_spectral,
        'Modularity KMeans': modularity_kmeans,
        'Modularity Spectral': modularity_spectral
    }




# Function to convert PyTorch tensors to NumPy arrays
def tensor_to_numpy(tensor):
    if isinstance(tensor, torch.Tensor):
        return tensor.detach().numpy()
    return tensor

# Updated function for clustering and evaluation
def cluster_and_evaluate_tensor(embeddings, graph):
    # Convert embeddings to NumPy if they are PyTorch tensors
    embeddings = tensor_to_numpy(embeddings)

    # Normalize embeddings for better clustering performance
    normalized_embeddings = normalize(embeddings)

    # Clustering: K-Means
    kmeans = KMeans(n_clusters=5, random_state=42)
    kmeans_labels = kmeans.fit_predict(normalized_embeddings)

    # Clustering: Spectral Clustering
    spectral = SpectralClustering(n_clusters=5, affinity='nearest_neighbors', random_state=42)
    spectral_labels = spectral.fit_predict(normalized_embeddings)

    # Calculate Silhouette Score
    silhouette_kmeans = silhouette_score(normalized_embeddings, kmeans_labels)
    silhouette_spectral = silhouette_score(normalized_embeddings, spectral_labels)

    # Modularity Calculation
    def calculate_modularity(graph, labels):
        # Create a dictionary to group nodes by their labels
        communities = {i: [] for i in set(labels)}
        for node, label in enumerate(labels):
            communities[label].append(node)

        # Convert to a list of sets for modularity calculation
        community_list = [set(community) for community in communities.values()]

        # Check if the communities form a valid partition
        all_nodes = set(graph.nodes)
        covered_nodes = set().union(*community_list)
        if all_nodes != covered_nodes:
            raise ValueError("Generated communities do not cover all nodes in the graph.")

        return nx.algorithms.community.modularity(graph, community_list)

    modularity_kmeans = calculate_modularity(graph, kmeans_labels)
    modularity_spectral = calculate_modularity(graph, spectral_labels)

    return {
        'Silhouette KMeans': silhouette_kmeans,
        'Silhouette Spectral': silhouette_spectral,
        'Modularity KMeans': modularity_kmeans,
        'Modularity Spectral': modularity_spectral
    }


In [ ]:
# Assuming `G` is your graph (NetworkX Graph object)

# 1. Laplacian Eigenmaps
L = nx.laplacian_matrix(G).toarray()
n_components = min(L.shape[0], 64)
pca = PCA(n_components=n_components)
laplacian_embeddings = pca.fit_transform(L)

# 2. HOPE (High-Order Proximity Embedding)
A = nx.to_numpy_array(G)
nmf = NMF(n_components=n_components, init='random', random_state=42)
hope_embeddings = nmf.fit_transform(A)

# 3. DeepWalk
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=4)
deepwalk_model = node2vec.fit(window=10, min_count=1, batch_words=4)
deepwalk_embeddings = {node: deepwalk_model.wv[node] for node in G.nodes()}

# 4. node2vec (Biased Random Walks)
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, p=1, q=1, workers=4)
node2vec_model = node2vec.fit(window=10, min_count=1, batch_words=4)
node2vec_embeddings = {node: node2vec_model.wv[node] for node in G.nodes()}

# 5. Struc2Vec (Simulated for illustration)
struc2vec_embeddings = np.random.rand(len(G.nodes()), 64)


# Output the embeddings for each method
print("Laplacian Eigenmaps Embeddings: ", laplacian_embeddings.shape)
print("HOPE Embeddings: ", hope_embeddings.shape)
print("DeepWalk Embeddings: ", len(deepwalk_embeddings), " nodes")
print("node2vec Embeddings: ", len(node2vec_embeddings), " nodes")
print("Struc2Vec (Simulated) Embeddings: ", struc2vec_embeddings.shape)

In [ ]:
# Run clustering and evaluation for each set of embeddings
results_laplacian = cluster_and_evaluate(laplacian_embeddings, G)
results_hope = cluster_and_evaluate(hope_embeddings, G)
results_deepwalk = cluster_and_evaluate(list(deepwalk_embeddings.values()), G)
results_node2vec = cluster_and_evaluate(list(node2vec_embeddings.values()), G)
results_struc2vec = cluster_and_evaluate(struc2vec_embeddings, G)


# Output results
print("Laplacian Eigenmaps Clustering Results:", results_laplacian)
print("HOPE Clustering Results:", results_hope)
print("DeepWalk Clustering Results:", results_deepwalk)
print("node2vec Clustering Results:", results_node2vec)
print("Struc2Vec (Simulated) Clustering Results:", results_struc2vec)

In [ ]:
G_cora = to_networkx(data, to_undirected=True)
G = G_cora

# 1. GCN (Graph Convolution Network)
class GCN(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 64)
        self.conv2 = GCNConv(64, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

gcn = GCN(in_channels=data.x.shape[1], out_channels=64)
gcn_embeddings = gcn(data)

# 2. GraphSAGE (Graph Sample and Aggregate)
class GraphSAGE(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, 64)
        self.conv2 = SAGEConv(64, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

graphsage = GraphSAGE(in_channels=data.x.shape[1], out_channels=64)
graphsage_embeddings = graphsage(data)

# 3. GAT (Graph Attention Network)
class GAT(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, 64, heads=8)
        self.conv2 = GATConv(64 * 8, out_channels, heads=1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

gat = GAT(in_channels=data.x.shape[1], out_channels=64)
gat_embeddings = gat(data)

# 4. DGI (Deep Graph Infomax)
class DGI(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(DGI, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.fc = nn.Linear(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.fc(x)
        return x

dgi = DGI(in_channels=data.x.shape[1], hidden_channels=64, out_channels=64)
dgi_embeddings = dgi(data)

# Visualization of embeddings (using PCA for visualization)
def plot_embeddings(embeddings, title):
    embeddings = embeddings.detach().numpy()
    pca = PCA(n_components=2)
    reduced_embeddings = pca.fit_transform(embeddings)
    plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1])
    plt.title(title)
    plt.show()

# # Plotting embeddings for each method
# plot_embeddings(gcn_embeddings, "GCN")
# plot_embeddings(graphsage_embeddings, "GraphSAGE")
# plot_embeddings(gat_embeddings, "GAT")
# plot_embeddings(dgi_embeddings, "DGI")

# Output the embeddings for each method
print("GCN Embeddings: ", gcn_embeddings.shape)
print("GraphSAGE Embeddings: ", graphsage_embeddings.shape)
print("GAT Embeddings: ", gat_embeddings.shape)
print("DGI Embeddings: ", dgi_embeddings.shape)

In [ ]:

# Run clustering and evaluation for each set of embeddings
results_gcn = cluster_and_evaluate_tensor(gcn_embeddings, G)
results_graphsage = cluster_and_evaluate_tensor(graphsage_embeddings, G)
results_gat = cluster_and_evaluate_tensor(gat_embeddings, G)
results_dgi = cluster_and_evaluate_tensor(dgi_embeddings, G)

# Output results
print("GCN Clustering Results:", results_gcn)
print("GraphSAGE Clustering Results:", results_graphsage)
print("GAT Clustering Results:", results_gat)
print("DGI Clustering Results:", results_dgi)

In [ ]:
# Convert graph to adjacency matrix
adj = nx.to_numpy_array(G)

# Normalize the adjacency matrix
scaler = StandardScaler()
adj_scaled = scaler.fit_transform(adj)

# Define the Autoencoder model for SDNE
input_layer = Input(shape=(adj.shape[1],))
encoded = Dense(64, activation='relu')(input_layer)  # Embedding layer
decoded = Dense(adj.shape[1], activation='sigmoid')(encoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer=Adam(), loss='mean_squared_error')

# Train the Autoencoder
autoencoder.fit(adj_scaled, adj_scaled, epochs=50, batch_size=16, shuffle=True, verbose=1)

# Get the encoded embeddings (lower-dimensional representation)
encoder = Model(input_layer, encoded)
sdne_embeddings = encoder.predict(adj_scaled)

# Visualize the embeddings using PCA
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(sdne_embeddings)
plt.figure(figsize=(8, 6))
plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1])
plt.title("SDNE Embeddings (Karate Club)")
plt.show()

print(f"SDNE Embeddings shape: {sdne_embeddings.shape}")

In [ ]:
results_sdne = cluster_and_evaluate(sdne_embeddings,G)
print("SDNE Clustering Results:", results_sdne)

In [ ]:
# Compute k-hop adjacency matrices (e.g., for k = 1, 2, 3)
adj_k1 = adj
adj_k2 = np.matmul(adj, adj)  # k=2 (second-order proximity)
adj_k3 = np.matmul(adj_k2, adj)  # k=3 (third-order proximity)

# Normalize each k-hop adjacency matrix to stabilize NMF
adj_k1 /= adj_k1.max()
adj_k2 /= adj_k2.max()
adj_k3 /= adj_k3.max()

# Stack all k-hop matrices
adj_matrices = [adj_k1, adj_k2, adj_k3]
combined_adj = np.hstack(adj_matrices)

# Apply Non-negative Matrix Factorization (NMF) for embedding
nmf = NMF(n_components=64, init='random', random_state=42, max_iter=200)
embeddings = nmf.fit_transform(combined_adj)

# Visualize the embeddings using PCA
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embeddings)
plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], c=list(G.nodes))
plt.title("GraRep Embeddings (Karate Club)")
plt.colorbar(label="Node Index")
plt.show()

print(f"GraRep Embeddings shape: {embeddings.shape}")

In [ ]:
results_GraRep = cluster_and_evaluate(embeddings,G)
print("GraRep Clustering Results:", results_GraRep)

In [ ]:
# Convert adjacency matrix to torch tensor
adj_tensor = torch.tensor(adj, dtype=torch.float32)

# Define the Graph Autoencoder model
class GraphAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(GraphAutoencoder, self).__init__()
        self.encoder = nn.Linear(input_dim, hidden_dim)
        self.decoder = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        z = F.relu(self.encoder(x))  # Encoding step
        return torch.sigmoid(self.decoder(z))  # Decoding step

# Initialize model, optimizer and loss function
model = GraphAutoencoder(input_dim=adj_tensor.shape[0], hidden_dim=64)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(1000):
    optimizer.zero_grad()
    output = model(adj_tensor)
    loss = F.binary_cross_entropy(output, adj_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Get the embeddings (encoded representations)
embeddings = model.encoder(adj_tensor).detach().numpy()

In [ ]:
results_G_AE = cluster_and_evaluate(embeddings,G)
print("Graph Autoencoder Clustering Results:", results_G_AE)

In [ ]:
# Define the VGAE model with an additional decoder layer
class VariationalGraphAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, output_dim):
        super(VariationalGraphAutoencoder, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, latent_dim)
        self.decoder = nn.Linear(latent_dim, output_dim)  # Decoder layer to reconstruct original features

    def encode(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

    def forward(self, x, edge_index):
        z = self.encode(x, edge_index)
        return self.decoder(z)  # Reconstruct node features

# Initialize model, optimizer and loss function
model = VariationalGraphAutoencoder(input_dim=dataset.num_node_features, hidden_dim=64, latent_dim=64, output_dim=dataset.num_node_features)
optimizer = optim.Adam(model.parameters())

# Training loop
for epoch in range(1000):
    model.train()
    optimizer.zero_grad()
    reconstructed_x = model(data.x, data.edge_index)
    loss = F.mse_loss(reconstructed_x, data.x)  # Reconstruct node features
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Get the embeddings (encoded representations)
embeddings = model.encode(data.x, data.edge_index).detach().numpy()

In [ ]:
results_VAGE = cluster_and_evaluate(embeddings,G)
print("Graph Autoencoder Decoder Clustering Results:", results_VAGE)

In [ ]:
# Define Poincaré embedding model (using a custom library)
class PoincareEmbedding(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(PoincareEmbedding, self).__init__()
        self.embedding = nn.Embedding(input_dim, latent_dim)

    def forward(self, x):
        return self.embedding(x)

# Example usage for a small graph or tree
input_dim = G_cora.number_of_nodes()
latent_dim = 64
model = PoincareEmbedding(input_dim, latent_dim)

# Random input nodes
nodes = torch.LongTensor([i for i in range(input_dim)])

# Forward pass to get embeddings
embeddings = model(nodes)

# Apply PCA to reduce embeddings to 2D for visualization
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings.detach().numpy())

# Visualize in 2D
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1])
plt.title("Poincaré Embeddings (PCA Reduced to 2D)")
plt.show()

# Print embeddings shape
print("Embeddings Shape: ", embeddings.shape)

In [ ]:
results_Poincare = cluster_and_evaluate_tensor(embeddings,G)
print("Poincaré Results:", results_Poincare)